In [32]:
#Importações
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc
spark = SparkSession.builder.appName("covid").getOrCreate();

In [33]:
df = spark.read.csv('covid_19_data.csv', inferSchema=True, header=True)

In [34]:
type(df)

pyspark.sql.dataframe.DataFrame

In [35]:
#Importando sql
from pyspark.sql import SQLContext

In [36]:
# createOrReplaceTempView -> criar uma visão temporária com o nome myTable (visão permite acessar os dados como uma tabela SQL)
df.createOrReplaceTempView("coronaTable")

In [39]:
#df.filter(df['Province/State'].isNull()).count()
#df.na.fill('0',subset=['Deaths']).show()
sql = spark.sql("select * from coronaTable where `Province/State` is null")
sql.show()

+---+---------------+--------------+--------------+---------------+---------+------+---------+
|SNo|ObservationDate|Province/State|Country/Region|    Last Update|Confirmed|Deaths|Recovered|
+---+---------------+--------------+--------------+---------------+---------+------+---------+
| 36|     01/22/2020|          null|         Japan|1/22/2020 17:00|      2.0|   0.0|      0.0|
| 37|     01/22/2020|          null|      Thailand|1/22/2020 17:00|      4.0|   0.0|      2.0|
| 38|     01/22/2020|          null|   South Korea|1/22/2020 17:00|      1.0|   0.0|      0.0|
| 75|     01/23/2020|          null|         Japan|  1/23/20 17:00|      1.0|   0.0|      0.0|
| 76|     01/23/2020|          null|      Thailand|  1/23/20 17:00|      4.0|   0.0|      2.0|
| 77|     01/23/2020|          null|   South Korea|  1/23/20 17:00|      1.0|   0.0|      0.0|
| 78|     01/23/2020|          null|     Singapore|  1/23/20 17:00|      1.0|   0.0|      0.0|
| 79|     01/23/2020|          null|   Philippines

In [40]:
#preencher com valores 0 a coluna 'Deaths' quando houver valor null
df.na.fill(0,subset=['Deaths']).show(10)

+---+---------------+--------------+--------------+---------------+---------+------+---------+
|SNo|ObservationDate|Province/State|Country/Region|    Last Update|Confirmed|Deaths|Recovered|
+---+---------------+--------------+--------------+---------------+---------+------+---------+
|  1|     01/22/2020|         Anhui|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  2|     01/22/2020|       Beijing|Mainland China|1/22/2020 17:00|     14.0|   0.0|      0.0|
|  3|     01/22/2020|     Chongqing|Mainland China|1/22/2020 17:00|      6.0|   0.0|      0.0|
|  4|     01/22/2020|        Fujian|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  5|     01/22/2020|         Gansu|Mainland China|1/22/2020 17:00|      0.0|   0.0|      0.0|
|  6|     01/22/2020|     Guangdong|Mainland China|1/22/2020 17:00|     26.0|   0.0|      0.0|
|  7|     01/22/2020|       Guangxi|Mainland China|1/22/2020 17:00|      2.0|   0.0|      0.0|
|  8|     01/22/2020|       Guizhou|Mainland China

In [41]:
sql.show(10)

+---+---------------+--------------+--------------+---------------+---------+------+---------+
|SNo|ObservationDate|Province/State|Country/Region|    Last Update|Confirmed|Deaths|Recovered|
+---+---------------+--------------+--------------+---------------+---------+------+---------+
| 36|     01/22/2020|          null|         Japan|1/22/2020 17:00|      2.0|   0.0|      0.0|
| 37|     01/22/2020|          null|      Thailand|1/22/2020 17:00|      4.0|   0.0|      2.0|
| 38|     01/22/2020|          null|   South Korea|1/22/2020 17:00|      1.0|   0.0|      0.0|
| 75|     01/23/2020|          null|         Japan|  1/23/20 17:00|      1.0|   0.0|      0.0|
| 76|     01/23/2020|          null|      Thailand|  1/23/20 17:00|      4.0|   0.0|      2.0|
| 77|     01/23/2020|          null|   South Korea|  1/23/20 17:00|      1.0|   0.0|      0.0|
| 78|     01/23/2020|          null|     Singapore|  1/23/20 17:00|      1.0|   0.0|      0.0|
| 79|     01/23/2020|          null|   Philippines

In [42]:
#importando when e col para processar os dados da coluna flag
from pyspark.sql.functions import when

In [43]:
# criando outro DataFrame (sobrescrevendo ) o DataFrame anterior, alteramos a coluna 'Province/State' quando ela possuísse valor null para 'N/A', caso contrário, retornamos o valor já existente
df = df.withColumn( 'Province/State', when ( col ('Province/State').isNull() , 'N/A' ).otherwise(df['Province/State'])   )

In [44]:
# filtrando a coluna 'Province/State' para verificar se ainda possui valores nulos
df.filter ( df['Province/State'].isNull() ).show(1)

+---+---------------+--------------+--------------+-----------+---------+------+---------+
|SNo|ObservationDate|Province/State|Country/Region|Last Update|Confirmed|Deaths|Recovered|
+---+---------------+--------------+--------------+-----------+---------+------+---------+
+---+---------------+--------------+--------------+-----------+---------+------+---------+



In [45]:
df.printSchema()

root
 |-- SNo: integer (nullable = true)
 |-- ObservationDate: string (nullable = true)
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Last Update: string (nullable = true)
 |-- Confirmed: double (nullable = true)
 |-- Deaths: double (nullable = true)
 |-- Recovered: double (nullable = true)



In [46]:
df.columns

['SNo',
 'ObservationDate',
 'Province/State',
 'Country/Region',
 'Last Update',
 'Confirmed',
 'Deaths',
 'Recovered']

In [47]:
df.show(3)

+---+---------------+--------------+--------------+---------------+---------+------+---------+
|SNo|ObservationDate|Province/State|Country/Region|    Last Update|Confirmed|Deaths|Recovered|
+---+---------------+--------------+--------------+---------------+---------+------+---------+
|  1|     01/22/2020|         Anhui|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  2|     01/22/2020|       Beijing|Mainland China|1/22/2020 17:00|     14.0|   0.0|      0.0|
|  3|     01/22/2020|     Chongqing|Mainland China|1/22/2020 17:00|      6.0|   0.0|      0.0|
+---+---------------+--------------+--------------+---------------+---------+------+---------+
only showing top 3 rows



In [48]:
#Ranking dos paises com mais mortes
fdgp = df.groupBy("Country/Region").max()
fdgp.sort( col ("max(Deaths)").desc()).select(["Country/Region","max(Deaths)"]).show(10)

+--------------+-----------+
|Country/Region|max(Deaths)|
+--------------+-----------+
|            UK|   112182.0|
|        France|   104093.0|
|        Brazil|    97058.0|
|          Iran|    72484.0|
|         India|    69615.0|
|        Poland|    68068.0|
|     Argentina|    64252.0|
|            US|    62078.0|
|  South Africa|    54417.0|
|     Indonesia|    45796.0|
+--------------+-----------+
only showing top 10 rows



In [49]:
#Pais com mais infectados
dfCountry = df.groupBy("Country/Region").max()
dfCountry.sort(col("max(Confirmed)").desc()).select(["Country/Region","max(Confirmed)"]).show(10)

+--------------+--------------+
|Country/Region|max(Confirmed)|
+--------------+--------------+
|        France|     5605532.0|
|        Turkey|     4875388.0|
|         India|     4665754.0|
|            UK|     3861901.0|
|            US|     3744830.0|
|     Argentina|     3005259.0|
|        Brazil|     2923367.0|
|        Poland|     2803233.0|
|          Iran|     2534855.0|
|     Indonesia|     1677274.0|
+--------------+--------------+
only showing top 10 rows



In [50]:
#Pais com mais recuperados
dfCountry2 = df.groupBy("Country/Region").max()
dfCountry2.sort(col("max(Recovered)").desc()).select(["Country/Region","max(Recovered)"]).show(10)

+--------------+--------------+
|Country/Region|max(Recovered)|
+--------------+--------------+
|            US|     6399531.0|
|        Turkey|     4480381.0|
|         India|     3930302.0|
|     Argentina|     2676197.0|
|        Brazil|     2588973.0|
|        Poland|     2520968.0|
|          Iran|     1988165.0|
|        Mexico|     1869529.0|
|          Peru|     1748170.0|
|Czech Republic|     1555215.0|
+--------------+--------------+
only showing top 10 rows



In [51]:
#Imports
from pyspark.sql.functions import year
from pyspark.sql.functions import month
from pyspark.sql.functions import dayofmonth

In [52]:
from pyspark.sql.functions import unix_timestamp, to_date

In [53]:
df = df.withColumn('ObservationDate',to_date(unix_timestamp(col("ObservationDate"), 'MM/dd/yyyy').cast('timestamp')))

In [54]:
df.show(5)

+---+---------------+--------------+--------------+---------------+---------+------+---------+
|SNo|ObservationDate|Province/State|Country/Region|    Last Update|Confirmed|Deaths|Recovered|
+---+---------------+--------------+--------------+---------------+---------+------+---------+
|  1|     2020-01-22|         Anhui|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  2|     2020-01-22|       Beijing|Mainland China|1/22/2020 17:00|     14.0|   0.0|      0.0|
|  3|     2020-01-22|     Chongqing|Mainland China|1/22/2020 17:00|      6.0|   0.0|      0.0|
|  4|     2020-01-22|        Fujian|Mainland China|1/22/2020 17:00|      1.0|   0.0|      0.0|
|  5|     2020-01-22|         Gansu|Mainland China|1/22/2020 17:00|      0.0|   0.0|      0.0|
+---+---------------+--------------+--------------+---------------+---------+------+---------+
only showing top 5 rows



In [55]:
df.select(year(df['ObservationDate'])).show(10)

+---------------------+
|year(ObservationDate)|
+---------------------+
|                 2020|
|                 2020|
|                 2020|
|                 2020|
|                 2020|
|                 2020|
|                 2020|
|                 2020|
|                 2020|
|                 2020|
+---------------------+
only showing top 10 rows



In [69]:
#Ano com mais mortes
df = df.withColumn("Year",year(df["ObservationDate"]))
dfm = df.groupBy("Year").max()
dfm.select(["Year","max(Deaths)"]).show()

+----+-----------+
|Year|max(Deaths)|
+----+-----------+
|2020|    64267.0|
|2021|   112182.0|
+----+-----------+



In [64]:
#Mes com mais mortes
df = df.withColumn("Month",month(df["ObservationDate"]))
dfmo = df.groupBy("Month").max()
dfmo.sort(col("max(Deaths)").desc()).select(["max(Year)","max(month)","DayodMonth","max(Deaths)"]).show()

+---------+----------+----------+-----------+
|max(Year)|max(month)|DayodMonth|max(Deaths)|
+---------+----------+----------+-----------+
|     2021|        12|         2|   112182.0|
|     2021|        12|         1|   112169.0|
|     2021|        12|        30|   112163.0|
|     2021|        12|        29|   112149.0|
|     2021|        12|        28|   112134.0|
|     2021|        12|        27|   112107.0|
|     2021|        12|        26|   112091.0|
|     2021|        12|        25|   112088.0|
|     2021|        12|        24|   112081.0|
|     2021|        12|        23|   112054.0|
|     2021|        12|        22|   112016.0|
|     2021|        12|        21|   112005.0|
|     2021|        12|        20|   111985.0|
|     2021|        12|        19|   111957.0|
|     2021|        12|        18|   111955.0|
|     2021|        12|        17|   111948.0|
|     2021|        12|        16|   111917.0|
|     2021|        12|        15|   111887.0|
|     2021|        12|        14| 

In [27]:
#Dias com mais mortes

df = df.withColumn("DayofMonth",dayofmonth(df["ObservationDate"]))
dfd = df.groupBy("DayofMonth").max()
dfd.sort(col("max(Deaths)").desc()).select(["max(Year)","max(month)","DayofMonth","max(Deaths)"]).show()

+----------+-----------+
|DayofMonth|max(Deaths)|
+----------+-----------+
|        31|   111489.0|
|        28|   112134.0|
|        26|   112091.0|
|        27|   112107.0|
|        12|   111810.0|
|        22|   112016.0|
|         1|   112169.0|
|        13|   111828.0|
|         6|   111626.0|
|        16|   111917.0|
|         3|   111585.0|
|        20|   111985.0|
|         5|   111611.0|
|        19|   111957.0|
|        15|   111887.0|
|         9|   111761.0|
|        17|   111948.0|
|         4|   111595.0|
|         8|   111710.0|
|        23|   112054.0|
+----------+-----------+
only showing top 20 rows



In [39]:
#Provincia com mais mortes
dfpro = df.groupBy("Province/State").max()
dfpro.sort(col("max(Deaths)").desc()).select(["Province/State","max(Deaths)"]).show(10)

+--------------+-----------+
|Province/State|max(Deaths)|
+--------------+-----------+
|       England|   112182.0|
|           N/A|   104093.0|
|     Sao Paulo|    97058.0|
|   Maharashtra|    69615.0|
|    California|    62078.0|
|      New York|    52358.0|
|         Texas|    50290.0|
|Rio de Janeiro|    44835.0|
|       Florida|    35268.0|
|  Minas Gerais|    34289.0|
+--------------+-----------+
only showing top 10 rows



In [43]:
sql1 = spark.sql("select distinct `Province/State` from coronaTable").show()

+--------------------+
|      Province/State|
+--------------------+
|                Utah|
|           Cajamarca|
|             Antwerp|
|              Hawaii|
|               Pasco|
|     Vastra Gotaland|
|Riverside County, CA|
|            Manitoba|
|            Anguilla|
|            Arequipa|
|   Dagestan Republic|
|       Kaluga Oblast|
|Unknown Location, MA|
|               Kochi|
| Okaloosa County, FL|
|            Nagaland|
|   Washoe County, NV|
|  El Paso County, CO|
|           Karnataka|
|        Cundinamarca|
+--------------------+
only showing top 20 rows



In [78]:
dfGroupBy = df.groupBy('Year','Month','Country/Region' ).sum()

In [80]:
dfGroupBy.sort(col('Year').desc(),col('Month').desc(),col('sum(Deaths)').desc()).show(5)

+----+-----+--------------+--------+--------------+-----------+--------------+---------+----------+---------------+
|Year|Month|Country/Region|sum(SNo)|sum(Confirmed)|sum(Deaths)|sum(Recovered)|sum(Year)|sum(Month)|sum(DayodMonth)|
+----+-----+--------------+--------+--------------+-----------+--------------+---------+----------+---------------+
|2021|    5|            US|33018698|   6.4813808E7|  1153767.0|           0.0|   234436|       580|            174|
|2021|    5|        Brazil|15369662|   2.9480885E7|   814076.0|   2.6107442E7|   109134|       270|             81|
|2021|    5|        Mexico|18786036|     4696653.0|   434401.0|     3736720.0|   133386|       330|             99|
|2021|    5|         India|21061384|   3.9114914E7|   431084.0|   3.1984542E7|   149554|       370|            111|
|2021|    5|            UK| 9107032|     8869988.0|   255578.0|       29368.0|    64672|       160|             48|
+----+-----+--------------+--------+--------------+-----------+---------